# Data Cleaning 1. Removing False Matches (Person Tags)

Data cleaning will comprise of three phases:
1. Removing false matches (tags that do not refer to an entity at all)
1. Correcting miscategorized labels (tags that misinterpret the entity category, ex. assign "PER" to a location)
1. Correcting truncated or extended spans (tags which are correct but whose extent is wrong, incorporating irrelevant tokens or ignoring relevant ones).
1. Identifying untagged entities

Perfect tagging might be impossible, but we can improve model results significantly by taking these measures.
This is an iterative process that will improve results as we go through the phases.

## Strategy 1. Capitalized names
    Duncan Kinkead capitalized all artist names in his documents, so any tags that have picked up capitalized strings are true names. 
    Any strings that are the lowercase equivalent of those capitalized names are also person names.
    
## Strategy 2 .First and last name lists
    We have developed a list of first and last names from 1400-1800 from a historical biographical dictionary. Using these, we can accept that any tags that incorporate a token from these lists refer to an actual person.
    
    Because we do not want to be to broad, we will accept tokens with a first name AND last name token that correspond to a word in the list. 
    Strings that do not include both a first name and a last name will have to be manually checked, which includes strings that have only one token. 
    
## Strategy 3.  Fuzzy Matching

## Strategy 4 Visual check of remaining person tags in CSV

## Strategy 5 Visual check of remaining tags against documents

Visual check of remaining person tags
    If strategies 1 and 2 discard enough PER tags, check the other ones manually.
    
## Strategy 4. Removing literary and religious references
    Some documents refer to ahistorical persons. This is especially important because the names of certain locations and associations take the names of Saints and other religious figures. (move this to "correcting miscategorized labels")
    


In [2]:
import pandas as pd
import re
import string

In [4]:
tagsDF = pd.read_csv('../Text Mining (NER)/EntitiesEMSModel2.csv')

In [5]:
tagsDF.head()

,Unnamed: 0,docid,string,label,start,end
0,0,1,martin de gaynça,PER,1,4
1,1,1,santa iglesia,ORG,13,15
2,2,1,Sevilla,LOC,16,17
3,3,1,juan sanchez de caliz,PER,18,22
4,4,1,sevilla,LOC,34,35


In [54]:
tagsDF.groupby('label').count()

,Unnamed: 0,docid,string,start,end
label,,,,,
DATE,6001,6001,6001,6001,6001
LOC,16578,16578,16578,16578,16578
MON,10365,10365,10365,10365,10365
ORG,5778,5778,5778,5778,5778
PER,33120,33120,33120,33120,33120


# 1. Create a list of PER tags with IDS

We are creating a list of person tags with their unique ids. We remove stop words, excess whitespace and punctuation to reduce variation between strings. 

In [4]:
# Test: PER tags
a = tagsDF.loc[tagsDF["label"] == "PER"] #select PER tags
b = a.sort_values('string') #sort alphabetically
#b['string'] = b['string'].str.lower()  #lowercase all strings

More data cleaning: removing punctuation and accents

In [5]:
people=b

people.head()

,Unnamed: 0,docid,string,label,start,end
57935,57935,6792,"""""""",PER,135,138
45285,45285,4978,- Rodríguez,PER,6,8
45173,45173,4961,-- Gómez de Brito,PER,34,38
45108,45108,4952,-- Manrique,PER,11,13
61780,61780,7342,-- Ramírez,PER,29,31


# 2 Clean strings to remove stopwords and accents

In [6]:
remove = string.punctuation
remove = remove.replace("-", "")

In [7]:
def clean_iter (line):
    result = re.sub(' del ', ' ', line) #remove stopwords
    result = re.sub(' de ',' ',result)
    result = re.sub(' la ',' ',result)
    
    #result = re.sub(\s\s, \s, result) #remove extra spaces
    
    result= re.sub('á','a',result) #remove accents
    result= re.sub('é','e',result)
    result= re.sub('í','i',result)
    result= re.sub('ó','o',result)
    result= re.sub('ú','u',result)
    result= re.sub('ü','u',result)
    result= re.sub('à','a',result)
    result= re.sub('è','e',result)
    result= re.sub('ì','i',result)
    result= re.sub('ò','o',result)
    result= re.sub('ù','u',result)
    result= re.sub('Á','A',result)
    result= re.sub('É','E',result)
    result= re.sub('Í','I',result)
    result= re.sub('Ó','O',result)
    result= re.sub('Ú','U',result)
    result= re.sub('Ü','U',result)
    result= re.sub('À','A',result)
    result= re.sub('È','E',result)
    result= re.sub('Ì','I',result)
    result= re.sub('Ò','O',result)
    result= re.sub('Ù','U',result)
    result= result.strip()
    
    #result = result.translate(str.maketrans('', '', string.punctuation)) #remove punctuation
    table = str.maketrans('', '', remove)
    result = result.translate(table)
    
    return result
    
clean_series=[]

for index, row in people.iterrows():
    clean_series.append(clean_iter(row['string']))
people['string']= clean_series


In [8]:
people.head()

,Unnamed: 0,docid,string,label,start,end
57935,57935,6792,,PER,135,138
45285,45285,4978,- Rodriguez,PER,6,8
45173,45173,4961,-- Gomez Brito,PER,34,38
45108,45108,4952,-- Manrique,PER,11,13
61780,61780,7342,-- Ramirez,PER,29,31


# 3 Create an attribute for checking process called 'Remove'

Then we insert a column for a dummy variable for "remove" or "keep". We will use this to mark the false tags that have to be removed with a 1. Tags that should be kept will have a 0 value. Blank values must be double-checked manually before being assigned a 0 or 1.

In [9]:
people.insert(6,"Remove","")

In [10]:
people.head()

,Unnamed: 0,docid,string,label,start,end,Remove
57935,57935,6792,,PER,135,138,
45285,45285,4978,- Rodriguez,PER,6,8,
45173,45173,4961,-- Gomez Brito,PER,34,38,
45108,45108,4952,-- Manrique,PER,11,13,
61780,61780,7342,-- Ramirez,PER,29,31,


Then we create a list of unique person tags.

In [11]:
uniquepers = people['string'].value_counts(sort=True)
uniquepers

juan martinez montañes    354
ALONSO PEREZ              268
andres ocanpo             206
JUAN DE VALDES            191
juan obiedo               191
                         ... 
Catalina Maria              1
Joseph Martinez             1
Gutierrez Machuca           1
baltasar becerra            1
diego hurtado               1
Name: string, Length: 14517, dtype: int64

There are 14517 unique strings.

# Strategy 1. Mark all Capitalized words and their lowecase equivalents as true names

Kinkead set all artist (painter, architect, goldbeater, gilder, etc) names in All-Caps. We can identify these strings and their lowercase equivalents as true names.

In [12]:
perscaps = people.loc[people['string'].str.isupper()==True]
perscaps.head()

,Unnamed: 0,docid,string,label,start,end,Remove
66787,66787,8056,AGUSTIN FRANCO,PER,7,9,
71698,71698,8579,AGUSTIN DE PEREA,PER,101,104,
45413,45413,4999,AGUSTIN DE PEREA,PER,16,19,
45421,45421,5000,AGUSTIN DE PEREA,PER,49,52,
45397,45397,4997,AGUSTIN DE PEREA,PER,26,29,


We can get an idea of what these strings are with a unique list.

In [13]:
uniqueperscaps = perscaps['string'].value_counts(sort=True)
uniqueperscaps

ALONSO PEREZ           268
JUAN DE VALDES         191
BARTOLOME MURILLO      176
CARLOS DE ZARATE       166
MATIAS DE ARTEAGA      119
                      ... 
JOAN DE ARRENAS          1
JUHAN MOL                1
DIEGO DE LAS CASAS       1
PEDRO DE OSORIO          1
FRANCISCO DE BARONA      1
Name: string, Length: 892, dtype: int64

In [14]:
UPCapslist = list(uniqueperscaps.index.values)

Using this list of unique capitalized strings, we can set Remove='0' for any strings that match an uppercase name.

In [15]:
for ind in people.index:
    if people.loc[ind,'string'] in UPCapslist:
        people.loc[ind,'Remove']=0

In [16]:
grouped = people.groupby('Remove')

grouped.count()

,Unnamed: 0,docid,string,label,start,end
Remove,,,,,,
,27101,27101,27101,27101,27101,27101
0,6019,6019,6019,6019,6019,6019


This process has marked 6019 instances of a true name, which should be equivalent to the artists found in Kinkead's Pintores y Doradores en Sevilla.

Now we mark any lowercase strings equivalent to the uppercase tags we know are correct as 0 as well:

In [17]:
for ind in people.index:
    if people.loc[ind,'string'].upper() in UPCapslist:
        people.loc[ind,'Remove']=0

In [38]:
grouped = people.groupby('Remove')
grouped.count()

,Unnamed: 0,docid,string,label,start,end
Remove,,,,,,
0,7065,7065,7065,7065,7065,7065
,26055,26055,26055,26055,26055,26055


Below is a shorter way of achieving the same goal:

In [36]:
perscaps = people['string'][people['string'].str.isupper()==True].unique().tolist()

In [37]:
people.loc[people['string'].str.upper().isin(perscaps),'Remove'] = 0

These approaches have identified a total 7065 tags as acceptable. That is, an additional 1046 tags corresponded to the same artists found in Kinkead. 

(Note: we will need to remove any documents that appear in Kinkead and other sources. This should be easy because Kinkead gives references as to where texts were published.)

# Strategy 2. Name Dictionary

In another notebook, we web scraped and cleaned the contents of the Spanish Royal History Academy's Spanish Biographical Dictionary (for persons born between 1400-1800). The process resulted in a list of first names and last names, which we revised to remove some problematic tokens (articles that would have picked up too many matches, as well as some words that appear often outside of naming). 

Let's load these in the notebook:

In [6]:
firstnames = open('DBE_Names/firstnames.txt','r')
lastnames = open('DBE_Names/lastnames.txt','r')

In [20]:
fnames = []
fnames = [line.rstrip('\n') for line in open('firstnames.txt')]

In [21]:
lnames = []
lnames = [line.rstrip('\n') for line in open('lastnames.txt')]

In [79]:
fnames

['abd',
 'abdo',
 'aben',
 'abraham',
 'abu',
 'abul',
 'acisclo',
 'adamo',
 'adan',
 'adaucto',
 'adria',
 'adriaen',
 'adrian',
 'adriano',
 'adrien',
 'advincula',
 'afan',
 'afonso',
 'agapito',
 'agesilao',
 'agostinho',
 'agostino',
 'agusti',
 'agustin',
 'agustina',
 'ahmad',
 'ahuitzotl',
 'aime-jacques',
 'al-basti',
 'al-garnati',
 'al-hayari',
 'al-malik',
 'al-manzari',
 'al-qalasadi',
 'al-titwani',
 'alal',
 'alberico',
 'albert-octave',
 'alberti',
 'alberto',
 'albino',
 'albret',
 'alcantara',
 'alcocer',
 'aldonza',
 'aleix',
 'aleixo',
 'alejandrino',
 'alejandro',
 'alejo',
 'alemania',
 'alessandro',
 'alex',
 'alexander',
 'alexandre',
 'alexo',
 'alexos',
 'alfonso',
 'ali',
 'almanzor',
 'almidez',
 'alonso',
 'alvar',
 'alvarado',
 'alvaro',
 'amadeo',
 'amador',
 'amalia',
 'amancio',
 'amaneo',
 'amaru',
 'ambrosia',
 'ambrosio',
 'amelia',
 'amerigo',
 'amparo',
 'ana',
 'anacaona',
 'anacleto',
 'anastasio',
 'andre',
 'andrea',
 'andres',
 'andresa',
 'a

In [177]:
lnames

['olim',
 'alava',
 'ab-ach',
 'abad',
 'abadal',
 'abadia',
 'abadiano',
 'abadie',
 'abadin',
 'abarca',
 'abaria',
 'abarrategui',
 'abascal',
 'abasolo',
 'abastas',
 'abat',
 'abbeville',
 'abd',
 'abduladin',
 'abel',
 'abella',
 'abellan',
 'abello',
 'aben',
 'abenaxara',
 'abengoechea',
 'abiancos',
 'abianelo',
 'ablanque',
 'ablitas',
 'aboab',
 'abollado',
 'aborim',
 'abraha',
 'abraham',
 'abranches',
 'abravanel',
 'abreu',
 'abrich',
 'abu',
 'abuin',
 'aburruza',
 'acacio',
 'acassuso',
 'accursio',
 'acebal',
 'acebedo',
 'acebes',
 'acebo',
 'acedo',
 'aceites',
 'acero',
 'acevedo',
 'acha',
 'achutegui',
 'acitores',
 'acorbe',
 'acorda',
 'acosta',
 'acquaviva',
 'acuña',
 'adalid',
 'adam',
 'adame',
 'adan',
 'adaro',
 'adarzo',
 'adell',
 'adeva',
 'adrada',
 'adrian',
 'adriano',
 'adsor',
 'aduart',
 'aduarte',
 'adurza',
 'aedo',
 'aerts',
 'afan',
 'afferden',
 'afonso',
 'aganduru',
 'agar',
 'agnesio',
 'agoiz',
 'agramont',
 'agramonte',
 'agraz',
 'agre

One issue is when certain names appear in both lists - then the string can have only one token belonging to both lists. 

But let's remove names that are repeated in the two lists. First we will make a list of the shared elements:

In [22]:
f = set(fnames)
l = set(lnames)

if (f & l): 
        f_and_l = list(f & l)
        print(f_and_l)

['borja', 'lopez', 'ordoño', 'justiniano', 'ali', 'ramiro', 'matias', 'rui', 'victor', 'agustin', 'rodriguez', 'leon', 'clemente', 'paullu', 'maximiliano', 'fabian', 'suero', 'austria', 'conrado', 'james', 'rubin', 'al-manzari', 'huascar', 'jacome', 'juste', 'arias', 'camaquiri', 'severino', 'corella', 'camilo', 'guerau', 'tito', 'tome', 'hernandillo', 'amadeo', 'angel', 'lope', 'fadrique', 'antonio', 'elio', 'mata', 'esteve', 'das', 'floris', 'morejon', 'bernardo', 'estefania', 'fortun', 'vidal', 'alcocer', 'rufo', 'afonso', 'madux', 'domingo', 'baltasar', 'yupanqui', 'muley', 'ortega', 'tomasa', 'claudio', 'clara', 'benito', 'aragon', 'nuño', 'asensio', 'marco', 'gregorio', 'beatriz', 'correia', 'jordan', 'ortuño', 'dalmau', 'joaquin', 'bernat', 'enriquez', 'gabriel', 'robert', 'antonia', 'cacamatzin', 'jeronimo', 'poechos', 'abd', 'fermin', 'ana', 'romeu', 'valeriano', 'rosa', 'obregon', 'abu', 'cayetano', 'toribio', 'mauro', 'pedroso', 'valentin', 'hurtado', 'teodoro', 'andres', 'z

And we will drop them all from the lnames list (our function does not actually make a semantic difference between both lists).

In [23]:
lnames = list(l-(f&l))

In [78]:
lnames

['lugo',
 'moxica',
 'jacas',
 'ocon',
 'mojica',
 'armenteros',
 'asso',
 'chover',
 'saraza',
 'peinado',
 'gariot',
 'tendilla',
 'isabe',
 'sangronis',
 'marquesado',
 'tamarid',
 'vial',
 'gaona',
 'gironella',
 'garcia-herreros',
 'macazaga',
 'casale',
 'conti',
 'buzo',
 'laloo',
 "d'aragona",
 'abrich',
 'tablares',
 'jofreu',
 'morante',
 'spannocchi',
 'bassadona',
 'pinar',
 'portu',
 'casedron',
 'bentallol',
 'bazcardo',
 'connoch',
 'hungri',
 'teruel',
 'masdovelles',
 'evreux',
 'piran',
 'xaso',
 'haedo',
 'horna',
 'fuenmayor',
 'lema',
 'meya',
 'burunda',
 'bollo',
 'sanclemente',
 'araucho',
 'bissech',
 'hovyne',
 'pezzi',
 'balaguer',
 'arbasia',
 'belloni',
 'castellvi',
 'conyngham',
 'ruedas',
 'carmitg',
 'tarrega',
 'eztenaga',
 'contero',
 'lencastre',
 'lucero',
 'creswell',
 'silva',
 'isniel',
 'comalada',
 'lastra',
 'moy',
 'gervete',
 'bocanegra',
 'seijas',
 'basilis',
 'paravicino',
 'ponzoa',
 'calar',
 'monteros',
 'conesa',
 'seneca',
 'ville',


Below: We create a function that results in True for any string that contains one token from each list, and apply it to the dataframe.

In [24]:
any_in = lambda a, b, c: any(i.lower() in b for i in a) and any(i.lower() in c for i in a) 

In [40]:
people['Remove'] = people.apply(lambda x: 0 if any_in(x['string'].split(),fnames,lnames) and len(x['string'].split())>1 else x['Remove'], axis=1)

The function above checks every row in the dataframe, assigning people['Remove'] = 0 whenever there is at least one token from the string in both the first name and last name lists. 

The added condition of length string.split() being greater than one at least weeds out names that have only one token. This is useful particularly for removing names of saints, likely from descriptions of paintings or images.



In [41]:
grouped = people.groupby('Remove')
grouped.count()

,Unnamed: 0,docid,string,label,start,end
Remove,,,,,,
0,20750,20750,20750,20750,20750,20750
,12370,12370,12370,12370,12370,12370


In [42]:
len(people['string'].unique())

14517

Number of unique strings identified:

In [43]:
len(people.loc[people['Remove']==0,'string'].unique())

7794

Number of unique strings not yet identified:

In [44]:
len(people.loc[people['Remove']=='','string'].unique())

6723

Let's export both lists of unique strings (confirmed and unconfirmed) to print them out, look at them and try to identify patterns for further work.

In [45]:
a= people.loc[people['Remove']==0,'string'].str.lower()
a= a.unique()
a= pd.Series(a)
len(a)

6752

In [46]:
b= people.loc[people['Remove']=='','string'].str.lower()
b= b.unique()
b= pd.Series(b)
len(b)

6271

In [47]:
a.to_csv("acceptedpersnames.csv")
b.to_csv("remainingpersnames.csv")

Will also export the intermediate people table to work with them in different sessions:

In [48]:
people.to_csv("Files_Cleaning1/people_strat2.csv")

# Strategy 3: Fuzzy Matching

Many of the unmatched tags above are variations of real names with approximate matches in the list.  We can use fuzzy matching to further find names.


In [49]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Generate a unique dataframe of tokens from our strings:

In [223]:
tokens = pd.DataFrame()

In [224]:
tokens['token']=''
tokens['score']=''
tokens['match']=''

In [225]:
tokens

,token,score,match


In [226]:
a = list(people['string'].str.split())

In [18]:
a

[[],
 ['Rodriguez'],
 ['Gomez', 'Brito'],
 ['Manrique'],
 ['Ramirez'],
 ['Sarabia'],
 ['Velas'],
 ['Velasco'],
 ['Herrera', 'Ortiz'],
 ['Pinta', 'y', 'Collantes'],
 ['Abrego'],
 ['Nino', 'Guevara'],
 ['de', 'Vega'],
 ['geronimo', 'hernandez'],
 ['luisa', 'ordoñes'],
 ['basco', 'perea'],
 ['gaspar', 'aguila'],
 ['vasco', 'pereyra'],
 ['gaspar', 'nuñes', 'delgado'],
 [],
 [],
 ['alonso', 'padilla'],
 ['las'],
 ['su'],
 ['A', 'Lopez'],
 ['AGUSTIN', 'FRANCO'],
 ['AGUSTIN', 'DE', 'PEREA'],
 ['AGUSTIN', 'DE', 'PEREA'],
 ['AGUSTIN', 'DE', 'PEREA'],
 ['AGUSTIN', 'DE', 'PEREA'],
 ['AGUSTIN', 'DE', 'PEREA'],
 ['AGUSTIN', 'FRANCO'],
 ['AGUSTIN', 'FRANCO'],
 ['AGUSTIN', 'FRANCO'],
 ['AGUSTIN', 'FRANCO'],
 ['AGUSTIN', 'FRANCO'],
 ['AGUSTIN', 'FRANCO'],
 ['AGUSTIN', 'GONZALEZ'],
 ['AGUSTIN', 'GONZALEZ'],
 ['AGUSTIN', 'PEREZ'],
 ['AGUSTIN', 'PEREZ'],
 ['AGUSTIN', 'PEREZ', 'ROMERO'],
 ['AGUSTIN', 'PEREZ', 'ROMERO'],
 ['AGUSTIN', 'PEREZ', 'ROMERO'],
 ['ALBERTO', 'JUANES'],
 ['ALFONSO', 'FAJARDO'],
 ['A

In [227]:
flat_a = [item for sublist in a for item in sublist]

In [228]:
set_a = set(flat_a)

In [229]:
for token in set_a:
    tokens= tokens.append({'token':token}, ignore_index=True)

In [230]:
len(tokens)

7289

In [231]:
tokens = tokens.replace(to_replace='None', value='')

Next, we are going to merge the first and last name lists.

In [232]:
names = fnames

names.extend(lnames)

Next, we will define a function that gives us the maximum value of the match

In [233]:
len(names)

10005

In [234]:
def fuzzmax(string,names):
        string = str(string).lower()
        result = list(map(lambda a: fuzz.token_sort_ratio(string,a),names))
        return [max(result),names[result.index(max(result))]]

In [235]:
type(fuzzmax("Antonio de Medina","Jose"))

list

In [236]:
fuzzmax("Antonio",['Jose','Manuel','Antonia'])

[86, 'Antonia']

We use this function to find the best matches for each token:

In [237]:
tokens['score'] = tokens.apply(lambda x: fuzzmax(x['token'], names), axis = 1)

In [238]:
tokens.head()

,token,score,match
0,BENJUMEA,"[71, enjuta]",NaN
1,Prevendado,"[71, revenga]",NaN
2,Lateral,"[77, antera]",NaN
3,MORILLO,"[100, morillo]",NaN
4,Ramon,"[100, ramon]",NaN


In [239]:
tokens['match']=tokens.apply(lambda x: x['score'][1],axis=1)

In [240]:
tokens['score']=tokens.apply(lambda x: x['score'][0], axis=1)

In [241]:
tokens.head()

,token,score,match
0,BENJUMEA,71,enjuta
1,Prevendado,71,revenga
2,Lateral,77,antera
3,MORILLO,100,morillo
4,Ramon,100,ramon


In [242]:
tokens.to_csv('token_match_scores.csv')

In [243]:
tokens[tokens['score']>90]

,token,score,match
3,MORILLO,100,morillo
4,Ramon,100,ramon
5,Ruela,91,iruñela
6,morante,100,morante
8,Coello,100,coello
...,...,...,...
7282,mendosa,92,mendoça
7283,carrion,100,carrion
7285,osuna,100,osuna
7286,lesmes,100,lesmes


In [50]:
tokens = pd.read_csv('token_match_scores.csv')

In [51]:
tokens.set_index('token', inplace=True)

In [52]:
tokens.index

Index(['BENJUMEA', 'Prevendado', 'Lateral', 'MORILLO', 'Ramon', 'Ruela',
       'morante', 'citacion', 'Coello', 'VELAZQUEZ',
       ...
       'nuño', 'cota', 'Infantes', 'mendosa', 'carrion', 'Ponze', 'osuna',
       'lesmes', 'Justino', 'REVES'],
      dtype='object', name='token', length=7289)

Function below:
    will split a string into tokens, and then add up their max scores from the tokens dataframe.
    We will divide by number of tokens to get the average match score, and accept any average match score above 90

In [53]:
def avgmatch(string):
    stringspl= string.split()
    scoresum=0
    if len(stringspl)>1:
        for token in stringspl:
            if token in tokens.index:
                a= tokens.loc[token,'score']
                scoresum=scoresum+a
        return(scoresum/len(stringspl))
    else:
        return 0

In [54]:
avgmatch("Manuel de Santiago")

93.33333333333333

In [55]:
people['Remove'] = people.apply(lambda x: 90 if (avgmatch(x['string'])>90 and x['Remove'])=='' else x['Remove'], axis=1)

In [56]:
grouped = people.groupby('Remove')
grouped.count()

,Unnamed: 0,docid,string,label,start,end
Remove,,,,,,
0,20750,20750,20750,20750,20750,20750
90,8937,8937,8937,8937,8937,8937
,3433,3433,3433,3433,3433,3433


In [57]:
a= people.loc[people['Remove']==0,'string'].str.lower()
a= a.unique()
a= pd.Series(a)
len(a)

6752

In [58]:
b= people.loc[people['Remove']==90,'string'].str.lower()
b= b.unique()
b= pd.Series(b)
len(b)

4094

In [59]:
c= people.loc[people['Remove']=='','string'].str.lower()
c= c.unique()
c= pd.Series(c)
len(c)

2177

In [60]:
b.to_csv("Files_Cleaning1/acceptedpersnames.csv")
c.to_csv("Files_Cleaning1/remainingpersnames.csv")

In [61]:
people.to_csv("Files_Cleaning1/people_strat3.csv")

# Strategy 4: Revision of Remaining unaccepted strings

Next, we went over these manually in excel, marking those that were clearly not names with a 1. We we also marked names that should be recategorized, and those that were correct, leaving blank only those that needed to be double checked against the document.

In [91]:
import pandas as pd
manual = pd.read_excel("Files_Cleaning1/RemainingPERS_HandTagged.xlsx")

In [92]:
manual.set_index('string',inplace=True)

In [93]:
manual.head()

,Unnamed: 0,Remove
string,,
NaN,0,NaN
- rodriguez,1,0
-- manrique,2,0
-- ramirez,3,0
-- sarabia,4,0


In [94]:
manual['Remove'] = manual['Remove'].astype(str)
manual['Remove'] = manual['Remove'].replace('nan','')

In [95]:
manual.groupby('Remove').count()

,Unnamed: 0
Remove,
,510
0,1371
1,269
D,1
L,11
M,4
O,11


Records tagged 0 (zero) were accepted through the exact match approach; those tagged 90 were accepted through fuzzy matching.

Records tagged O (letter o), L, D, M should be re-tagged as organizations, locations, dates, money respectively.

Records tagged 1 must be removed

And records without a remove tag must go on to the next phase of cleaning, checking against the documents.


Next, let's update the people DF with the manually added tags.

In [96]:
people = pd.read_csv("Files_Cleaning1/people_strat3.csv")

In [97]:
people['Remove'] = people['Remove'].astype(str)
people['Remove'] = people['Remove'].replace('nan','')
people['Remove'] = people['Remove'].replace('0.0','0')
people['Remove'] = people['Remove'].replace('90.0','90')

In [98]:
people.groupby('Remove').count()

,Unnamed: 0,Unnamed: 0.1,docid,string,label,start,end
Remove,,,,,,,
,3433,3433,3433,3430,3433,3433,3433
0,20750,20750,20750,20750,20750,20750,20750
90,8937,8937,8937,8937,8937,8937,8937


In [99]:
people['Remove'] = people.apply(lambda x: manual.loc[str(x['string']).lower(),'Remove'] if str(x['string']).lower() in manual.index else x['Remove'], axis=1)

In [100]:
grouped = people.groupby('Remove')
grouped.count()

,Unnamed: 0,Unnamed: 0.1,docid,string,label,start,end
Remove,,,,,,,
,830,830,830,827,830,830,830
0,22974,22974,22974,22974,22974,22974,22974
1,321,321,321,321,321,321,321
90,8937,8937,8937,8937,8937,8937,8937
D,1,1,1,1,1,1,1
L,12,12,12,12,12,12,12
M,5,5,5,5,5,5,5
O,40,40,40,40,40,40,40


So we have 830 records remaining to mark for acceptance or removal. Of these, there are 510 distinct strings, which corresponds to the labels we could not identify manually.

In [101]:
rem= people.loc[people['Remove']=='','string'].str.lower()
rem= rem.unique()
rem= pd.Series(rem)
len(rem)

510

We also went ahead and checked the accepted names from both the exact and fuzzy matching with the name dictionary. Excitingly, the vast majority were good matches, but we marked a few as wrong or dubious.  

In [111]:
manual2 = pd.read_excel("AcceptedPERS_Checked.xlsx")
manual2.set_index('string',inplace=True)
manual2['Remove'] = manual2['Remove'].astype(str)
manual2['Remove'] = manual2['Remove'].replace('nan','')

In [119]:
manual2.groupby('Remove').count()

,Unnamed: 0
Remove,
,42
0,4038
1,11
D,1
L,2


In [112]:
people['Remove'] = people.apply(lambda x: manual2.loc[str(x['string']).lower(),'Remove'] if str(x['string']).lower() in manual2.index else x['Remove'], axis=1)

In [113]:
grouped = people.groupby('Remove')
grouped.count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,docid,string,label,start,end
Remove,,,,,,,,
,904,904,904,904,901,904,904,904
0,31820,31820,31820,31820,31820,31820,31820,31820
1,333,333,333,333,333,333,333,333
D,2,2,2,2,2,2,2,2
L,16,16,16,16,16,16,16,16
M,5,5,5,5,5,5,5,5
O,40,40,40,40,40,40,40,40


And just to double check, we have 904 records to revise, of which 552 are distinct strings. This corresponds to the 510 + 42 unaccepted strings from our two excel files. We have not lost any records in our dataset.

In [117]:
rem= people.loc[people['Remove']=='','string'].str.lower()
rem= rem.unique()
rem= pd.Series(rem)
len(rem)

552

In [118]:
len(people)

33120

And let's save our intermediate CSV for future use too.

In [120]:
people.to_csv("Files_Cleaning1/people_strat4.csv")

# Strategy 5: Check Against the Documents

The tags still not marked 0 have not been identified as names. We will look at them within the context of their documents to mark whether they are names or not.

In [ ]:
# For future reference: per tags up to this point have been saved as "people_strat4.csv" 

In [1]:
import pandas as pd
people = pd.read_csv("people_strat4.csv")

In [14]:
people['string'] = people['string'].astype(str)
people['Remove'] = people['Remove'].astype(str)
people['Remove'] = people['Remove'].replace('nan','')
people['Remove'] = people['Remove'].replace('0.0','0')

In [3]:
people

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,docid,string,label,start,end,Remove
0,0,0,57935,57935,6792,NaN,PER,135,138,
1,1,1,45285,45285,4978,- Rodriguez,PER,6,8,0
2,2,2,45173,45173,4961,-- Gomez Brito,PER,34,38,0
3,3,3,45108,45108,4952,-- Manrique,PER,11,13,0
4,4,4,61780,61780,7342,-- Ramirez,PER,29,31,0
...,...,...,...,...,...,...,...,...,...,...
33115,33115,33115,45897,45897,5065,Angela Guillenes,PER,19,21,0
33116,33116,33116,65927,65927,7947,Angela Rodriguez Borte,PER,28,31,0
33117,33117,33117,11101,11101,1304,çamora,PER,238,239,0
33118,33118,33118,32475,32475,3436,çamora,PER,90,91,0


In [6]:
grouped = people.groupby('Remove')
grouped.count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,docid,string,label,start,end
Remove,,,,,,,,,
,904,904,904,904,904,901,904,904,904
0,31820,31820,31820,31820,31820,31820,31820,31820,31820
1,333,333,333,333,333,333,333,333,333
D,2,2,2,2,2,2,2,2,2
L,16,16,16,16,16,16,16,16,16
M,5,5,5,5,5,5,5,5,5
O,40,40,40,40,40,40,40,40,40


There are 1478 tags that do not contain at least one first name and one last name token from the lists above.  Next, we will go through them individually, looking at them in context, to determine whether they should be removed or kept.

In [7]:
from IPython.display import clear_output
import spacy
from spacy import displacy

In [8]:
import pickle

file = open("../Text Mining (NER)/ Trained_EMS2_NER_data.p", 'rb')
docs = pickle.load(file)

In [127]:
docs[1]

(...martin de gaynça... soy convenido... con los reverendos padres prior frailes e convento del monasterio de san pablo desta dicha ciudad de Sevilla... en tal manera que yo me obligo de fazer el cruzero de la iglesia junto a la capilla mayor... e de lo fazer de buena obra de canteria del puerto de santa maria del altura de la capilla mayor lo cual tengo de derrocar a mi costa con tanto que todo e material que saliere del dicho cruzero asi de piedra como de ladrillo sea mio e me obligo de ensanchar las ventanas del dicho cruzero del altura que fuere menester e desgarrar lo que fuere menester para ello e de encalar e torar los testeros donde estan las dichas ventanas (muy roto) e mas me obligo de fazer el pilarote sobre que a de venir el canpanario fasta en raz de la capilla e mas me obligo de alinpiar los dos arcos para la parte de la capilla que estan en el dicho cruzero... e de lo dar acabado en fin del mes de marzo de 1543... dandome 400 ducados,
 {'id': 2})

In [131]:
type(docs[1][0])

spacy.tokens.doc.Doc

With the code below, I iterate through the rows of the people dataframe, finding those where remove is still blank. For these, I show a span of the document with all entities highlighted (10 tokens after the string). I can input 0, 1 or a letter representing the new tag. If this isn't enough, typing X allows me to view the whole document, after which I can input the choice.

In [47]:
for (index, row) in people.iterrows():
    if row['Remove']=='':
        print(row['string'])
        
        docid = row['docid']
        strstart = int(row['start'])
        fetchend = strstart + 10
        
        for doc in docs:
            if doc[1]['id'] == docid:      
                displacy.render(doc[0][strstart:fetchend],style='ent',jupyter=True)
                people.loc[index, 'Remove']=input('Remove?')
                
                if people.loc[index, 'Remove']=='X':
                    for doc in docs:
                        if doc[1]['id'] == docid:
                            displacy.render(doc[0],style='ent',jupyter=True)
                            people.loc[index,'Remove']=input('Remove?')
                clear_output(wait=True)

Alonso


Remove? X


Remove? 0


In [48]:
grouped = people.groupby('Remove')
grouped.count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,docid,string,label,start,end
Remove,,,,,,,,,
0,32442,32442,32442,32442,32442,32442,32442,32442,32442
1,561,561,561,561,561,561,561,561,561
D,6,6,6,6,6,6,6,6,6
L,39,39,39,39,39,39,39,39,39
M,6,6,6,6,6,6,6,6,6
O,66,66,66,66,66,66,66,66,66


In [49]:
people.to_csv("Files_Cleaning1/people_strat5.csv")

Ultimately, we removed 561 tags and reclassified a total of 127.

The final step is to apply these changes to the relevant places: the people['label'] column and also the document which has the spacy tags.  DATE LOC MON ORG

In [51]:
people.groupby('label').count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,docid,string,start,end,Remove
label,,,,,,,,,
PER,33120,33120,33120,33120,33120,33120,33120,33120,33120


In [59]:
people.loc[people['Remove']=='D','label']= 'DATE'
people.loc[people['Remove']=='L','label']= 'LOC'
people.loc[people['Remove']=='M','label']= 'MON'
people.loc[people['Remove']=='O','label']= 'ORG'
people.loc[people['Remove']=='1','label']= ''

In [60]:
people.groupby('label').count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,docid,string,start,end,Remove
label,,,,,,,,,
,561,561,561,561,561,561,561,561,561
DATE,6,6,6,6,6,6,6,6,6
LOC,39,39,39,39,39,39,39,39,39
MON,6,6,6,6,6,6,6,6,6
ORG,66,66,66,66,66,66,66,66,66
PER,32442,32442,32442,32442,32442,32442,32442,32442,32442


In [61]:
people.to_csv("Files_Cleaning1/PER_tags_clean1.csv")

In [ ]:
#We will have to remember to combine these with the file that has all tag types (not just 'PER').